In [1]:

import DatabaseApi
import pandas as pd


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:

def getTeams():
        teams = DatabaseApi.getList('Tm','Players')[0]
        mod_teams=[]
        for team in teams:
            mod_teams.append(team[0])
        return mod_teams
    
def populatePlayers():
    for player_key in data_dict['player']:
        
        teams = getTeams()
        
        if player_key in teams:
            
            players= DatabaseApi.getStats('Player','Players','Tm',player_key)[0]
            for p in players:
                data_dict["player"].append(p[-1])
    
    data_dict['player'] = [val for val in data_dict['player'] if val not in teams]
    
def getAvgStats(gt="G"):
    ret=[]
    player_data_dict={}
    result={}
    for player_key in data_dict['player']:

        #db_data = DatabaseApi.getStats('*','Players','player',player_key)
        select_v = ', '.join(['AVG(\"{}\")'.format(item) if item[0].isdigit() else 'AVG({})'.format(item) for item in data_dict['stats']])
        db_data  = DatabaseApi.getBoxScoreStats(select_v,'BoxScores',['Player','BoxScore_Type'], [player_key,gt],None,'AVG') 
        if db_data == 0:
            continue
        player_data_dict[player_key] = dict(zip(db_data[1],db_data[0][0]))
        result[player_key]  = player_data_dict[player_key]
        #result[player_key] = { key: player_data_dict[player_key][key] for key in data_dict['stats'] if key in player_data_dict[player_key]}
        ret.append([player_key,result[player_key]])
        result={}
    return ret

def getXGames(x):
    ret=[]
    result={}
  
    player_data_dict={}
    for player_key in data_dict["player"]:
        
        for gt in data_dict['filter']['GameType']:
            special = {'AVG':{},'SUM':{}}
            
            for i in data_dict['stats']:
                if '(' in i and i[:i.index('(')] in special.keys():
                    special[i[:i.index('(')]][i[i.index('(')+1:i.index(')')]]=0
                    data_dict['stats'][data_dict['stats'].index(i)]= i[i.index('(')+1:i.index(')')]
                
            select_v = ', '.join(['\"{}\"'.format(item) if item[0].isdigit() else '{}'.format(item) for item in data_dict['stats']])
            select_v += ', Opponent'
            num = int(x[1:])-1
            db_data  = DatabaseApi.getBoxScoreStats(select_v,'BoxScores',['Player','BoxScore_Type'], [player_key,gt],num,'DESC' if x[0]=='L' else 'ASC') 
            if db_data == 0:
                continue
            if num >len(db_data[0])-1:
                num = len(db_data[0])-1
            i=0
            av = 0
            while i<=num:
                
                player_data_dict[player_key] = dict(zip(db_data[1],db_data[0][i]))
                result[player_key+" VS "+player_data_dict[player_key]["Opponent"]+" "+x[0]+str(i+1)] = { key: player_data_dict[player_key][key] for key in data_dict['stats'] if key in player_data_dict[player_key]}
                if db_data[0][i][0] == None:
                    i=i+1
                    av+=1
                    continue
                for st in db_data[1]:
                    for y in special:
                        if st in special[y]:
                            special[y][st]+=player_data_dict[player_key][st]              

                i=i+1
            
            
            for st in special['AVG']:
                    if (int(x[1:]) - av) != 0:
                        special["AVG"][st]/= int(x[1:]) - av
        
        ret.append([player_key,special["AVG"]])
            
    return ret

In [6]:

data_dict = {'player': [], 'team': [], 'stats': [], 'schedule':[],
                  
                  'filter':{ 
                      'GameType':[],
                      'XGames':[],
                      'DateRange':[],
                      'Date':[],
                      'PlayerOrTeam':[]
                  }, 
                  
                  }
special={}
data_dict["player"] = getTeams()
populatePlayers()
players=data_dict["player"]
mapping = []
for player in players:
    data_dict["player"] = [player]
    data_dict["stats"] = ['PTS+TRB+AST']
    data_dict["filter"]["GameType"] = ['G']
    t=getAvgStats()
    if t!=[]:
        mapping.append(getAvgStats()[0]) 
    

data_dict = {'player': [], 'team': [], 'stats': [], 'schedule':[],
                  
                  'filter':{ 
                      'GameType':[],
                      'XGames':[],
                      'DateRange':[],
                      'Date':[],
                      'PlayerOrTeam':[]
                  }, 
                  
                  }
special={}
data_dict["player"] = getTeams()
populatePlayers()

players=data_dict["player"]
for player in players:
    data_dict["player"] = [player]
    data_dict["stats"] = ['AVG(PTS+TRB+AST)']
    data_dict["filter"]["XGames"] =['L5']
    data_dict["filter"]["GameType"] = ['G']
    
    if t!=[]:
        mapping.append(getXGames("L5")[0])



In [7]:
compare= {}

for item in mapping:
    if item[0] in compare:
        compare[item[0]].update(item[1])
    else:
        compare[item[0]] = item[1]


    

In [9]:
df = pd.DataFrame.from_dict(compare).T
df["DIFF"] = df["PTS+TRB+AST"]-(df["AVG(PTS+TRB+AST)"])
df = df.sort_values(by=["DIFF"],ascending=False)
df.head(20)




,AVG(PTS+TRB+AST),PTS+TRB+AST,DIFF
Eugene Omoruyi,7.650000,29.000000,21.350000
Brandon Miller,22.674419,36.400000,13.725581
Donte DiVincenzo,18.220000,31.200000,12.980000
Jonathan Kuminga,21.760870,34.200000,12.439130
Stephen Curry,37.477273,49.400000,11.922727
Jordan Nwora,9.666667,20.500000,10.833333
Aaron Nesmith,18.063830,28.800000,10.736170
Vince Williams Jr.,16.552632,26.333333,9.780702
Cam Thomas,26.775000,36.000000,9.225000
Ayo Dosunmu,14.395833,23.600000,9.204167
